In [2]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from scipy.sparse import hstack,vstack
from sklearn.linear_model import LogisticRegression
import taskALoader as t_a
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV,ShuffleSplit
from sklearn.model_selection import PredefinedSplit
from langdetect import detect
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
from sklearn.utils import shuffle

In [3]:
#man_anntd = pd.read_csv("manually_annotated_risk.csv").fillna(' ')
man_anntd = pd.read_csv("manually_annotated_risk_2.csv").fillna(' ')

In [4]:
#man_anntd = man_anntd.sample(frac=1).reset_index(drop=True)

In [5]:
#man_anntd = man_anntd.head(247)

In [6]:
lab = man_anntd['risk_labels']
lab = list(lab)
lab.count(0)/float(len(lab))

0.5277310924369748

In [7]:
X = list(man_anntd['list_of_posts'])
y = list(man_anntd['risk_labels'])
y = np.asarray(y, dtype=np.float32)

In [8]:
print(128/(128+119))

0.5182186234817814


In [9]:

X_train_all, X_test, y_train_all, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train_all, y_train_all, test_size=0.25, random_state=1)

In [10]:
print(len(X_train),len(X_test))

357 119


In [11]:
corpus_df = pd.read_csv("clpsych19_training_data/shared_task_posts.csv").fillna(' ')
p_title = corpus_df['post_title']
p_body = corpus_df['post_body']
corpus_text = pd.concat([p_title,p_body]) 

In [12]:
print(type(p_body[100]))

<class 'str'>


In [13]:
p_body[1385]

'Do I have to look for manual for it? Buy a new one? Or what?'

In [14]:
'''
c = 0
for i,body in tqdm(enumerate(p_body)):
    #print(i)
    text = p_title[i]+" "+body
    if(len(text.strip())!=0):
        if(detect(str(text))!='en'):
            print(text,detect(text))
            c = c + 1
            
'''

'\nc = 0\nfor i,body in tqdm(enumerate(p_body)):\n    #print(i)\n    text = p_title[i]+" "+body\n    if(len(text.strip())!=0):\n        if(detect(str(text))!=\'en\'):\n            print(text,detect(text))\n            c = c + 1\n            \n'

In [15]:
def get_predefined_split(features_train, features_val, target_train, target_val):
    always_train = np.empty((target_train.shape[0]), dtype=np.int32)
    #print(target_train.shape[0])
    always_train[:] = -1
    always_validate = np.empty((target_val.shape[0]), dtype=np.int32)
    always_validate[:] = 0
    pred_split_indices = np.concatenate((always_train, always_validate))
    curr_target_train = np.concatenate((target_train, target_val))
    #curr_features_train = vstack((features_train, features_val))
    curr_features_train = np.concatenate((features_train, features_val))
   
    ps = PredefinedSplit(test_fold=pred_split_indices)

    return curr_features_train, curr_target_train, ps

In [16]:
class TfidfVectorizerWrapper(TfidfVectorizer):
    def fit(self,x,y=None, **fit_params):
        x = corpus_text
        return super(TfidfVectorizerWrapper, self).fit(x, y, **fit_params)
        
    def transform(self, x, y=None, **fit_params):
        #x = [content.split('\t')[0] for content in x]  # filtering the input
        return super(TfidfVectorizerWrapper, self).transform(x, y, **fit_params)       

In [17]:
def logistic_param_selection(X_train,X_val,y_train,y_val):
    

    
    linear_pipeline = Pipeline(
        [
            ('tfidf',
             TfidfVectorizerWrapper(decode_error='ignore', use_idf=True, ngram_range=(1, 2), lowercase=True,
                             stop_words='english',
                             analyzer='word', max_features=40000)),
            ('classifier', LogisticRegression(class_weight='balanced',max_iter=2000))]
    )

    
    
    ngram_range=[(1, 2), (1, 3), (1, 4), (1, 5)]
    #max_features=[100000, 200000, 300000, 400000, 500000]
    max_features = [200000,500000,600000,700000]
    min_df=[1, 2, 3]
    
    
    
    X,y,ps = get_predefined_split (X_train,X_val,y_train,y_val)
    
    #Logistic Regression hyperparameters
    #penalty = ['l2']
    penalty = ['l1','l2']
    Cs = [0.001, 0.01, 0.1, 1, 10]
    #multi_class = ['multinomial','ovr']
    multi_class = ['auto']
    #solver = ['newton-cg', 'lbfgs', 'sag']
    solver = ['liblinear']
    param_grid = {"tfidf__ngram_range": ngram_range,
                  "tfidf__use_idf": [True],
                  "tfidf__max_features": max_features,
                  "tfidf__min_df": min_df,
                  "tfidf__sublinear_tf": [True],
                  "classifier__C": Cs,
                  "classifier__multi_class":multi_class,
                  "classifier__solver":solver,
                  "classifier__penalty":penalty
                 
                 }
                             
    
   

    grid_search = GridSearchCV(linear_pipeline, param_grid,cv=ps,scoring='f1_macro', refit=False)

    clf=grid_search.fit(X, y)
 
    
    grid_search.best_params_
    return grid_search.best_params_,clf

In [18]:
import warnings
warnings.filterwarnings('ignore')
best_pars,model = logistic_param_selection(X_train,X_val,y_train,y_val)

In [19]:
def runBestLogistic(train,train_labels,test,test_labels,best_pars):
    
    
    c = best_pars['classifier__C']
    multi_class = best_pars['classifier__multi_class']
    penalty = best_pars['classifier__penalty']
    solver = best_pars['classifier__solver']
    
    
    max_features = best_pars['tfidf__max_features']
    min_df = best_pars['tfidf__min_df']
    ngram_range=best_pars['tfidf__ngram_range']
    sublinear_tf = best_pars['tfidf__sublinear_tf']
    use_idf = best_pars['tfidf__use_idf']
    
    
    
    linear_pipeline = Pipeline(
        [
            ('tfidf',
             TfidfVectorizerWrapper(decode_error='ignore', 
                                    ngram_range=ngram_range, lowercase=True,stop_words='english',
                             analyzer='word', max_features=max_features,min_df=min_df,
                                    sublinear_tf=sublinear_tf,use_idf=use_idf)),
            
            ('classifier', LogisticRegression(C=c,penalty = penalty,solver=solver,multi_class=multi_class,class_weight='balanced',max_iter=2000))]
    )
    
    
    
    
    #clf = LinearSVC(C=0.01, multi_class='crammer_singer',class_weight='balanced')
    #clf = LogisticRegression()
    #clf = SVC()
    #clf = linear_pipeline()
    #clf.fit(train, train_labels)
    linear_pipeline.fit(train,train_labels)
    predicted=linear_pipeline.predict(test)
    print(classification_report(test_labels, predicted))
    print ("Accuracy: {}".format(accuracy_score(test_labels, predicted)))
    return linear_pipeline

In [20]:
print(best_pars)

{'classifier__C': 10, 'classifier__multi_class': 'auto', 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear', 'tfidf__max_features': 200000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}


In [21]:
model = runBestLogistic(X_train,y_train,X_val,y_val,best_pars)
runBestLogistic(X_train,y_train,X_test,y_test,best_pars)

              precision    recall  f1-score   support

         0.0       0.88      0.80      0.84        65
         1.0       0.78      0.87      0.82        54

   micro avg       0.83      0.83      0.83       119
   macro avg       0.83      0.84      0.83       119
weighted avg       0.84      0.83      0.83       119

Accuracy: 0.8319327731092437
              precision    recall  f1-score   support

         0.0       0.84      0.77      0.81        62
         1.0       0.77      0.84      0.81        57

   micro avg       0.81      0.81      0.81       119
   macro avg       0.81      0.81      0.81       119
weighted avg       0.81      0.81      0.81       119

Accuracy: 0.8067226890756303


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizerWrapper(analyzer='word', binary=False, decode_error='ignore',
            dtype=<class 'numpy.float64'>, encoding='utf-8',
            input='content', lowercase=True, max_df=1.0,
            max_features=200000, min_df=2, ngram_range=(1, 2), norm='l2',
            pr...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [25]:

model = runBestLogistic(X,y,X_val,y_val,best_pars)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        65
         1.0       1.00      1.00      1.00        54

   micro avg       1.00      1.00      1.00       119
   macro avg       1.00      1.00      1.00       119
weighted avg       1.00      1.00      1.00       119

Accuracy: 1.0


In [26]:
import pickle

filename = '605_Lb_logi_binary_clf_model_all_data.sav'

pickle.dump(model, open(filename, 'wb'))

